# Le test de l'implémentation Prolog

In [1]:
type var = VarT of string * int | VarL of string * int;;
type plist = PLCons of (term list) * term | Empty
and term = Var of var | Term of string * (term list) | PList of plist;;
type clause = Clause of term * (term list);;

Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads



type var = VarT of string * int | VarL of string * int


type plist = PLCons of term list * term | Empty
and term = Var of var | Term of string * term list | PList of plist


type clause = Clause of term * term list


In [2]:
type token = TLPar | TRPar | TLBra | TRBra | TDot | TIf | TAnd | TVar of string | TAtom of string | TVert ;;

type token =
    TLPar
  | TRPar
  | TLBra
  | TRBra
  | TDot
  | TIf
  | TAnd
  | TVar of string
  | TAtom of string
  | TVert


In [3]:
(* Transforme une chaîne de caractères en liste et réciproquement *)
let to_list str = List.init (String.length str) (String.get str);;
let to_string lst = String.init (List.length lst) (List.nth lst);;


(* Enlève les espaces, tabulations de la liste de caractères *)
let rec remove_spaces = function
| (' ' | '\t' | '\n')::l -> remove_spaces l
| l -> l;;

(* Retire le premier mot (suite de lettres et de '_') de la liste *)
let rec read_word char_list mot = match char_list with
| ('A'..'Z' | 'a'..'z' | '_' as c)::l -> read_word l (c::mot)
| char_list -> char_list, to_string (List.rev mot);;

(* Pour séparer les variables et les atomes *)
let string_to_atomvar = function
| str -> (match String.get str 0 with
                | ('A'..'Z') -> TVar str
                | ('a'..'z') -> TAtom str
                | _ -> failwith "Wrong Word");;

(* Transforme la liste de caractères en liste de symboles *)
let rec charlist_to_tokenlist liste_car = let lst = remove_spaces liste_car in match lst with
| [] -> []
| '.'::l -> TDot::(charlist_to_tokenlist l)
| '('::l -> TLPar::(charlist_to_tokenlist l)
| ')'::l -> TRPar::(charlist_to_tokenlist l)
| '['::l -> TLBra::(charlist_to_tokenlist l)
| ']'::l -> TRBra::(charlist_to_tokenlist l)
| '|'::l -> TVert::(charlist_to_tokenlist l)
| ':'::'-'::l -> TIf::(charlist_to_tokenlist l)
| ','::l -> TAnd::(charlist_to_tokenlist l)
| ('A'..'Z' | 'a'..'z' | '_')::_ -> let l, m = read_word lst [] 
in (string_to_atomvar m)::(charlist_to_tokenlist l)
| l -> failwith ("Wrong Syntax in " ^ (to_string l));;

(* Transforme la chaîne de caractères en liste de symboles *)
let lexer str = charlist_to_tokenlist (to_list str);;

val to_list : string -> char list = <fun>


val to_string : char list -> string = <fun>


val remove_spaces : char list -> char list = <fun>


val read_word : char list -> char list -> char list * string = <fun>


val string_to_atomvar : string -> token = <fun>


val charlist_to_tokenlist : char list -> token list = <fun>


val lexer : string -> token list = <fun>


In [4]:
let program = "apprend(eve, mathematiques).
apprend(benjamin, informatique).
apprend(benjamin, physique).
enseigne(alice, physique).
enseigne(pierre, mathematiques).
enseigne(pierre, informatique).
etudiant_de(E,P):-apprend(E,M), enseigne(P,M).
test([a,b|A]).
";;

lexer program;;

val program : string =
  "apprend(eve, mathematiques).\napprend(benjamin, informatique).\napprend(benjamin, physique).\nenseigne(alice, physique).\nenseigne(pierre, mathematiques).\nenseigne(pierre, informatique).\netudiant_de(E,P):-apprend(E,M), enseigne(P,M).\ntest([a,b|A]).\n"


- : token list =
[TAtom "apprend"; TLPar; TAtom "eve"; TAnd; TAtom "mathematiques"; TRPar;
 TDot; TAtom "apprend"; TLPar; TAtom "benjamin"; TAnd; TAtom "informatique";
 TRPar; TDot; TAtom "apprend"; TLPar; TAtom "benjamin"; TAnd;
 TAtom "physique"; TRPar; TDot; TAtom "enseigne"; TLPar; TAtom "alice"; TAnd;
 TAtom "physique"; TRPar; TDot; TAtom "enseigne"; TLPar; TAtom "pierre";
 TAnd; TAtom "mathematiques"; TRPar; TDot; TAtom "enseigne"; TLPar;
 TAtom "pierre"; TAnd; TAtom "informatique"; TRPar; TDot;
 TAtom "etudiant_de"; TLPar; TVar "E"; TAnd; TVar "P"; TRPar; TIf;
 TAtom "apprend"; TLPar; TVar "E"; TAnd; TVar "M"; TRPar; TAnd;
 TAtom "enseigne"; TLPar; TVar "P"; TAnd; TVar "M"; TRPar; TDot;
 TAtom "test"; TLPar; TLBra; TAtom "a"; TAnd; TAtom "b"; TVert; TVar "A";
 TRBra; TRPar; TDot]


In [5]:
(* Sépare les différentes clauses *)
let extract_clauses token_list =
let rec f tk_lst tmp = match tk_lst with
| [] -> if tmp = [] then [] else failwith "No end of clause"
| TDot::l -> (List.rev tmp)::(f l [])
| t::l -> f l (t::tmp)
in f token_list [];;

extract_clauses (lexer program);;

val extract_clauses : token list -> token list list = <fun>


- : token list list =
[[TAtom "apprend"; TLPar; TAtom "eve"; TAnd; TAtom "mathematiques"; TRPar];
 [TAtom "apprend"; TLPar; TAtom "benjamin"; TAnd; TAtom "informatique";
  TRPar];
 [TAtom "apprend"; TLPar; TAtom "benjamin"; TAnd; TAtom "physique"; TRPar];
 [TAtom "enseigne"; TLPar; TAtom "alice"; TAnd; TAtom "physique"; TRPar];
 [TAtom "enseigne"; TLPar; TAtom "pierre"; TAnd; TAtom "mathematiques";
  TRPar];
 [TAtom "enseigne"; TLPar; TAtom "pierre"; TAnd; TAtom "informatique"; TRPar];
 [TAtom "etudiant_de"; TLPar; TVar "E"; TAnd; TVar "P"; TRPar; TIf;
  TAtom "apprend"; TLPar; TVar "E"; TAnd; TVar "M"; TRPar; TAnd;
  TAtom "enseigne"; TLPar; TVar "P"; TAnd; TVar "M"; TRPar];
 [TAtom "test"; TLPar; TLBra; TAtom "a"; TAnd; TAtom "b"; TVert; TVar "A";
  TRBra; TRPar]]


In [6]:
(* Sépare les membres de droite et de gauche d'une clause *)
let extract_rightleft cl =
    let rec f cl left = match cl with
    | [] -> (List.rev left), []
    | TIf::l -> (List.rev left), l
    | t::l -> f l (t::left)
in f cl [];;

extract_rightleft (lexer "etudiant_de(E,P):-apprend(E,M), enseigne(P,M)");;

val extract_rightleft : token list -> token list * token list = <fun>


- : token list * token list =
([TAtom "etudiant_de"; TLPar; TVar "E"; TAnd; TVar "P"; TRPar],
 [TAtom "apprend"; TLPar; TVar "E"; TAnd; TVar "M"; TRPar; TAnd;
  TAtom "enseigne"; TLPar; TVar "P"; TAnd; TVar "M"; TRPar])


In [7]:
(* Sépare les termes d'une liste de termes *)
let extract_term_list_old tk_lst =
let rec f tk_lst term_tmp lvl = match tk_lst with
    | [] -> [term_tmp]
    | TLPar::l -> f l (term_tmp @ [TLPar]) (lvl + 1)
    | TRPar::l -> if lvl > 0 then f l (term_tmp @ [TRPar]) (lvl - 1) 
                else failwith "Mismatched Parenthesis"
    | TAnd::l when lvl = 0 -> term_tmp::(f l [] 0)
    | t::l -> f l (term_tmp @ [t]) lvl
in f tk_lst [] 0;;

(* Sépare les termes d'une liste de termes *)
let extract_term_list tk_lst =
let rec f tk_lst term_tmp stack = match tk_lst with
    | [] -> [term_tmp]
    | TLPar::l -> f l (term_tmp @ [TLPar]) (TLPar::stack)
    | TRPar::l -> (match stack with
        | [] -> failwith "Mismatched Parenthesis"
        | TLPar::st -> f l (term_tmp @ [TRPar]) st
        | _ -> failwith "Mismatched Parenthesis")
    | TLBra::l -> f l (term_tmp @ [TLBra]) (TLBra::stack)
    | TRBra::l -> (match stack with
        | [] -> failwith "Mismatched Brackets"
        | TLBra::st -> f l (term_tmp @ [TRBra]) st
        | _ -> failwith "Mismatched Brackets")
    | TAnd::l when stack = [] -> term_tmp::(f l [] [])
    | t::l -> f l (term_tmp @ [t]) stack
in f tk_lst [] [];;

extract_term_list_old (lexer "apprend(E,M), enseigne(P,M)");;

extract_term_list (lexer "apprend([A,B,C]), enseigne([B,C,A]), [a(A),b(B),c(C) | [] ]");;

val extract_term_list_old : token list -> token list list = <fun>


val extract_term_list : token list -> token list list = <fun>


- : token list list =
[[TAtom "apprend"; TLPar; TVar "E"; TAnd; TVar "M"; TRPar];
 [TAtom "enseigne"; TLPar; TVar "P"; TAnd; TVar "M"; TRPar]]


- : token list list =
[[TAtom "apprend"; TLPar; TLBra; TVar "A"; TAnd; TVar "B"; TAnd; TVar "C";
  TRBra; TRPar];
 [TAtom "enseigne"; TLPar; TLBra; TVar "B"; TAnd; TVar "C"; TAnd; TVar "A";
  TRBra; TRPar];
 [TLBra; TAtom "a"; TLPar; TVar "A"; TRPar; TAnd; TAtom "b"; TLPar; TVar "B";
  TRPar; TAnd; TAtom "c"; TLPar; TVar "C"; TRPar; TVert; TLBra; TRBra; TRBra]]


In [8]:
(* Renvoie l'intérieur d'une liste de symboles dont les extrêmes sont des parenthèses *)
let inside_parenthesis = function
| [] -> failwith "Empty List"
| TLPar::l -> (let rec f = function
                | [] -> failwith "Mismatched Parenthesis"
                | [TRPar] -> []
                | h::t -> h::(f t)
                in f l)
| TLBra::l -> (let rec f = function
                | [] -> failwith "Mismatched Brackets"
                | [TRBra] -> []
                | h::t -> h::(f t)
                in f l)
| _ -> failwith "List is not beginning with Parenthesis or Bracket";;

inside_parenthesis (lexer "(E,M)");;
inside_parenthesis (lexer "[(E),[M],A]");;

val inside_parenthesis : token list -> token list = <fun>


- : token list = [TVar "E"; TAnd; TVar "M"]


- : token list =
[TLPar; TVar "E"; TRPar; TAnd; TLBra; TVar "M"; TRBra; TAnd; TVar "A"]


In [9]:
(* Sépare la droite et la gauche de la barre verticale | *)
let extract_vert tkl =
let rec f tkl tmp stack = match tkl with
| [] -> (List.rev tmp), []
| TLPar::l -> f l (TLPar::tmp) (stack+1)
| TLBra::l -> f l (TLBra::tmp) (stack+1)
| TRPar::l -> f l (TRPar::tmp) (stack-1)
| TRBra::l -> f l (TRBra::tmp) (stack-1)
| TVert::l when stack = 0 -> (List.rev tmp), l
| a::l -> f l (a::tmp) stack
in f tkl [] 0;;

val extract_vert : token list -> token list * token list = <fun>


In [10]:
(* Prend une liste de tokens qui est censée ne représenter qu'un terme 'mouton', 'élève(samuel)', 'X',
'apprend(X, Y)' et renvoie ce terme *)


let rec tokenlist_to_term = function
| [] -> failwith "Error : [] is not a term"
| (TVar str)::[] -> Var (VarT (str, 0))
| (TVar str)::_ -> failwith "No other arguments with Var"
| (TAtom str)::[] -> Term(str, [])
| (TAtom str)::l -> Term(str, List.map tokenlist_to_term (extract_term_list (inside_parenthesis l)))
| TLBra::TRBra::l -> PList Empty
| TLBra::l -> PList (tkl_to_plist (TLBra::l))
| _ -> failwith "Not a valid term"
and tkl_to_plist tkl = match extract_vert (inside_parenthesis tkl) with
| [], [] -> Empty
| l, [] -> PLCons ((List.map tokenlist_to_term (extract_term_list l)), PList Empty)
| g, [TVar str] -> PLCons ((List.map tokenlist_to_term (extract_term_list g)), (Var (VarL (str,0))))
| g, d -> PLCons ((List.map tokenlist_to_term (extract_term_list g)), PList (tkl_to_plist d));;

tokenlist_to_term (lexer "apprend(benjamin, informatique)");;
tokenlist_to_term (lexer "[benjamin, informatique, [A,B,C | T ]]");;

val tokenlist_to_term : token list -> term = <fun>
val tkl_to_plist : token list -> plist = <fun>


- : term =
Term ("apprend", [Term ("benjamin", []); Term ("informatique", [])])


- : term =
PList
 (PLCons
   ([Term ("benjamin", []); Term ("informatique", []);
     PList
      (PLCons
        ([Var (VarT ("A", 0)); Var (VarT ("B", 0)); Var (VarT ("C", 0))],
        Var (VarL ("T", 0))))],
   PList Empty))


In [11]:
(* Transforme une liste de symboles représentant une clause en la clause elle même *)
let tokenlist_to_clause tklcl = 
let c = extract_rightleft tklcl in let l = fst c and r = snd c
in Clause (tokenlist_to_term l, if r = [] then [] else List.map tokenlist_to_term (extract_term_list r));;

(* Transforme la chaîne de caractères du programme en la liste de clauses du format voulu *)
let parser str = List.map tokenlist_to_clause (extract_clauses (lexer str)) ;;

val tokenlist_to_clause : token list -> clause = <fun>


val parser : string -> clause list = <fun>


In [12]:
(* Un parser sur une chaîne représentant un terme *)
let parser_term str = tokenlist_to_term (lexer str);;

(* Un parser sur une chaîne représentant une liste de termes *)
let parser_term_list str = List.map tokenlist_to_term (extract_term_list (lexer str));;

(* Un parser sur une chaîne (sans le point) représentant une clause *)
let parser_clause str = tokenlist_to_clause (lexer str);;

parser_term "apprend(benjamin, informatique)";;

parser_term_list "apprend(E,M), enseigne(P,M)";;

parser_clause "etudiant_de(E,P):-apprend(E,M), enseigne(P,M)";;

val parser_term : string -> term = <fun>


val parser_term_list : string -> term list = <fun>


val parser_clause : string -> clause = <fun>


- : term =
Term ("apprend", [Term ("benjamin", []); Term ("informatique", [])])


- : term list =
[Term ("apprend", [Var (VarT ("E", 0)); Var (VarT ("M", 0))]);
 Term ("enseigne", [Var (VarT ("P", 0)); Var (VarT ("M", 0))])]


- : clause =
Clause (Term ("etudiant_de", [Var (VarT ("E", 0)); Var (VarT ("P", 0))]),
 [Term ("apprend", [Var (VarT ("E", 0)); Var (VarT ("M", 0))]);
  Term ("enseigne", [Var (VarT ("P", 0)); Var (VarT ("M", 0))])])


In [13]:
let world = parser program;;

val world : clause list =
  [Clause (Term ("apprend", [Term ("eve", []); Term ("mathematiques", [])]),
    []);
   Clause
    (Term ("apprend", [Term ("benjamin", []); Term ("informatique", [])]),
    []);
   Clause (Term ("apprend", [Term ("benjamin", []); Term ("physique", [])]),
    []);
   Clause (Term ("enseigne", [Term ("alice", []); Term ("physique", [])]),
    []);
   Clause
    (Term ("enseigne", [Term ("pierre", []); Term ("mathematiques", [])]),
    []);
   Clause
    (Term ("enseigne", [Term ("pierre", []); Term ("informatique", [])]), 
    []);
   Clause (Term ("etudiant_de", [Var (VarT ("E", 0)); Var (VarT ("P", 0))]),
    [Term ("apprend", [Var (VarT ("E", 0)); Var (VarT ("M", 0))]);
     Term ("enseigne", [Var (VarT ("P", 0)); Var (VarT ("M", 0))])]);
   Clause
    (Term ("test",
      [PList (PLCons ([Term ("a", []); Term ("b", [])], Var (VarL ("A", 0))))]),
    [])]


In [14]:
let rec term_to_string = function
| Var (VarT (str,n)) -> str
| Var (VarL (str,n)) -> str ^ "*"
(*| Var (Id (str,n)) -> str ^ (string_of_int n)*)
| Term (a, []) -> a
| Term (a, l) -> a ^ "(" ^ (String.concat ", " (List.map term_to_string l)) ^ ")"
| PList plst -> plist_to_string plst
and plist_to_string = function
| Empty -> "[]"
| PLCons (tl, PList Empty) -> "[" ^ (String.concat ", " (List.map term_to_string tl)) ^ "]"
| PLCons (tl, PList plst) -> "[" ^ (String.concat ", " (List.map term_to_string tl)) ^ " | " ^ (plist_to_string plst) ^ "]";;


let term_list_to_string l = String.concat ", " (List.map term_to_string l);;

let clause_to_string (Clause (h, l)) = (term_to_string h) ^ ":-" ^ (term_list_to_string l);;

File "[14]", line 8, characters 22-262:
 8 | ......................function
 9 | | Empty -> "[]"
10 | | PLCons (tl, PList Empty) -> "[" ^ (String.concat ", " (List.map term_to_string tl)) ^ "]"
11 | | PLCons (tl, PList plst) -> "[" ^ (String.concat ", " (List.map term_to_string tl)) ^ " | " ^ (plist_to_string plst) ^ "]"..
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
PLCons (_, (Var _|Term (_, _)))


val term_to_string : term -> string = <fun>
val plist_to_string : plist -> string = <fun>


val term_list_to_string : term list -> string = <fun>


val clause_to_string : clause -> string = <fun>


In [15]:
(* Teste si une variable est dans un terme *)
let rec var_in_term v = function
| Var w -> v = w
| Term (_, l) -> List.mem true (List.map (var_in_term v) l)
| PList Empty -> false
| PList (PLCons (l, t)) -> List.mem true (List.map (var_in_term v) l) || var_in_term v t;;

var_in_term (VarT ("P", 0)) (parser_term "[A,[B,Z],C | [E | P] ]");;

val var_in_term : var -> term -> bool = <fun>


- : bool = false


In [16]:
(* Remplace une variable par un nouveau terme, récursivement à l'intérieur d'un terme *)
let rec replace_var_in_term var new_t = function
| Var v when v = var -> new_t
| Var v -> Var v
| Term (a , l) -> Term (a, List.map (replace_var_in_term var new_t) l)
| PList Empty -> PList Empty
| PList (PLCons (l, t)) -> PList 
(PLCons ((List.map (replace_var_in_term var new_t) l), (replace_var_in_term  var new_t t)));;

replace_var_in_term (VarL ("E",0)) (parser_term "pierre") (parser_term "etudiant_de([E,B | E],P)");;

val replace_var_in_term : var -> term -> term -> term = <fun>


- : term =
Term ("etudiant_de",
 [PList
   (PLCons ([Var (VarT ("E", 0)); Var (VarT ("B", 0))], Term ("pierre", [])));
  Var (VarT ("P", 0))])


In [17]:
(* Teste si une variable est dans une liste de couples de termes *)
let var_in_eql v e = List.mem true (List.map (function t -> var_in_term v (fst t) || var_in_term v (snd t)) e);;

(* Remplace une variable par un nouveau terme, à l'intérieur d'une liste de couples de termes *)
let replace_var_in_eql var new_t eql = 
List.map (function (a, b) -> (replace_var_in_term var new_t a), (replace_var_in_term var new_t b)) eql;;

val var_in_eql : var -> (term * term) list -> bool = <fun>


val replace_var_in_eql :
  var -> term -> (term * term) list -> (term * term) list = <fun>


In [18]:
(* E est une liste de couples de termes *)
(* transforme E en un équivalent sous forme résolue (type option, None si impossible) *) 
let rec solve e =
    (* Parcourt une fois l'ensemble des équations *)
    let rec pass tmp = function
    | [] -> Some tmp
    
    | (Term (f, lf), Term (g, lg))::l when f = g && List.length lf = List.length lg -> 
        pass ((List.map2 (fun a b -> a,b) lf lg)@tmp) l
        
    | (Term (_, _), Term (_, _))::l -> None (* failwith "Unification failed" *)
    
    | (Var x, Var y)::l when x = y -> pass tmp l
    
    | (Term (a, la), Var x)::l -> pass ((Var x, Term (a, la))::tmp) l
    
    | (Var x, t)::l when var_in_eql x tmp || var_in_eql x l -> 
        if var_in_term x t then None (* failwith "Unification failed, loop" *)
        else pass ((Var x, t)::(replace_var_in_eql x t tmp)) (replace_var_in_eql x t l)
        
    | (Var x, t)::l -> pass ((Var x, t)::tmp) l
    
in match (pass [] e) with 
| None -> None
| Some eqlpp -> let b = List.sort_uniq compare eqlpp in if e = b then Some e else solve b;;

File "[18]", line 5, characters 23-712:
 5 | .......................function
 6 |     | [] -> Some tmp
 7 |     
 8 |     | (Term (f, lf), Term (g, lg))::l when f = g && List.length lf = List.length lg -> 
 9 |         pass ((List.map2 (fun a b -> a,b) lf lg)@tmp) l
...
18 |         if var_in_term x t then None (* failwith "Unification failed, loop" *)
19 |         else pass ((Var x, t)::(replace_var_in_eql x t tmp)) (replace_var_in_eql x t l)
20 |         
21 |     | (Var x, t)::l -> pass ((Var x, t)::tmp) l
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Term (_, _), PList _)::_|(PList _, _)::_)


val solve : (term * term) list -> (term * term) list option = <fun>


In [19]:
solve [parser_term "etudiant_de(E, P)", parser_term "etudiant_de(S, P)"];; (* E = S *)

- : (term * term) list option =
Some [(Var (VarT ("E", 0)), Var (VarT ("S", 0)))]


In [20]:
(* Prend 2 terme et renvoie l'unifieur le plus général s'il existe, None sinon. 
Attention, aucune variable ne doit être présente dans (r) la requête et dans (c) la tête de la clause *)

let rec mgu r c = let f = function
    | Var x, Term (a, l) -> x, Term (a, l)
    | Var x, Var y -> x, Var y
    | Term (_, _), _ -> failwith "Should not happen."
in match solve [r, c] with
| None -> None
| Some e -> Some (List.map f e);;

File "[20]", line 4, characters 26-162:
4 | ..........................function
5 |     | Var x, Term (a, l) -> x, Term (a, l)
6 |     | Var x, Var y -> x, Var y
7 |     | Term (_, _), _ -> failwith "Should not happen."
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((Var _, PList _)|(PList _, _))


val mgu : term -> term -> (var * term) list option = <fun>


In [21]:
(* La fonction test_unification a pour but de tester mgu *)
let test_unification str_r str_c = mgu (parser_term str_r) (parser_term str_c);;

test_unification "etudiant_de(E, pierre)" "etudiant_de(F,P)";; (* F/E et P/pierre *)

test_unification "etudiant_de(F,P)" "etudiant_de(E, pierre)";; (* E/F et P/pierre *)

test_unification "f(X,g(Y))" "f(g(Z),Z)";; (* X/g(g(Y)) et Z/g(Y) *)

test_unification "f(g(Z),Z)" "f(X,g(Y))";; (* X/g(g(Y)) et Z/g(Y) *)

val test_unification : string -> string -> (var * term) list option = <fun>


- : (var * term) list option =
Some
 [(VarT ("E", 0), Var (VarT ("F", 0))); (VarT ("P", 0), Term ("pierre", []))]


- : (var * term) list option =
Some
 [(VarT ("F", 0), Var (VarT ("E", 0))); (VarT ("P", 0), Term ("pierre", []))]


- : (var * term) list option =
Some
 [(VarT ("X", 0), Term ("g", [Term ("g", [Var (VarT ("Y", 0))])]));
  (VarT ("Z", 0), Term ("g", [Var (VarT ("Y", 0))]))]


- : (var * term) list option =
Some
 [(VarT ("X", 0), Term ("g", [Term ("g", [Var (VarT ("Y", 0))])]));
  (VarT ("Z", 0), Term ("g", [Var (VarT ("Y", 0))]))]


In [22]:
(* Applique une substitution sur un terme *)
let rec apply_subst_on_term uni term = match uni with
| [] -> term
| (v,t)::l -> apply_subst_on_term l (replace_var_in_term v t term);;

(* Applique une substitution sur une liste de termes *)
let rec apply_subst_on_termlist uni lst = List.map (apply_subst_on_term uni) lst;;

val apply_subst_on_term : (var * term) list -> term -> term = <fun>


val apply_subst_on_termlist : (var * term) list -> term list -> term list =
  <fun>


In [23]:
(* Prend un entier et une clause, renvoie la clause avec les variables renommées à l'entier *)
let rename n (Clause (t1, tl)) = 
let rec f = function 
| Var (VarT (str, _)) -> Var (VarT (str, n))
| Term (atm, l) -> Term (atm, List.map f l)
in Clause (f t1, List.map f tl);;

rename 42 (parser_clause "etudiant_de(E,P):-apprend(E,M), enseigne(P,M)");;

File "[23]", line 3, characters 12-110:
3 | ............function 
4 | | Var (VarT (str, _)) -> Var (VarT (str, n))
5 | | Term (atm, l) -> Term (atm, List.map f l)
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
(Var (VarL (_, _))|PList _)


val rename : int -> clause -> clause = <fun>


- : clause =
Clause (Term ("etudiant_de", [Var (VarT ("E", 42)); Var (VarT ("P", 42))]),
 [Term ("apprend", [Var (VarT ("E", 42)); Var (VarT ("M", 42))]);
  Term ("enseigne", [Var (VarT ("P", 42)); Var (VarT ("M", 42))])])


In [24]:
type 'a tree = Leaf of 'a | Node of (('a tree) Lazy.t) list;;

(* La fonction de recherche, renvoie l'arbre des solutions *)
let rec sld_tree world req subs n = match req with
| [] -> Leaf subs 
| head_request_term::other_request_terms -> 
    Node (List.filter_map (fun c -> let Clause (left_member, right_member) = rename n c in 
    (match mgu head_request_term left_member with
        | None -> None
        | Some unifier -> Some (lazy (sld_tree world 
            (apply_subst_on_termlist unifier (right_member@other_request_terms)) 
            (unifier::subs) (n+1)))
    )) world);;

type 'a tree = Leaf of 'a | Node of 'a tree Lazy.t list


val sld_tree :
  clause list ->
  term list -> (var * term) list list -> int -> (var * term) list list tree =
  <fun>


In [25]:
(* Compose correctement la liste de substitutions *)
let compose l = 
let compose2 s1 s2 = s1@(List.map (function (v2, lt2) -> (v2, apply_subst_on_term s1 lt2)) s2)
in List.fold_left compose2 [] l;;

(* Recherche les termes qui sont des variables récursivement dans une liste de termes *)
let rec find_vars_in_termlist tl = List.sort_uniq compare (List.concat (List.map 
(function 
| Var v -> [Var v]
| Term (_,l) -> find_vars_in_termlist l) 
tl));;

val compose : (var * term) list list -> (var * term) list = <fun>


File "[25]", line 8, characters 0-70:
 8 | (function 
 9 | | Var v -> [Var v]
10 | | Term (_,l) -> find_vars_in_termlist l).
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
PList _


val find_vars_in_termlist : term list -> term list = <fun>


In [26]:
let list_to_seq l = List.fold_right (fun x s -> (fun ()->Seq.Cons(x, s)) ) l Seq.empty;;

let rec to_seq = function
| Leaf str -> Seq.return str
| Node tl -> Seq.flat_map (fun par -> to_seq (Lazy.force par)) (list_to_seq tl);;

let solutions world req = let tree = sld_tree world (parser_term_list req) [] 1 in 
let vars = (find_vars_in_termlist (parser_term_list req)) in
Seq.map  (fun l -> vars, apply_subst_on_termlist (compose l) vars) (to_seq tree);;

let request world req = let sol = solutions world req in
if sol () = Seq.Nil then Format.printf "This is false.\n%!"
else Seq.iter (fun (vars, tl) -> 
if vars = [] then Format.printf "This is True.\n%!"
else Format.printf "There is : %s\n%!"
(String.concat ", " (List.map2 (fun v t -> (term_to_string v) ^ " = " ^ (term_to_string t)) vars tl))
) sol;;

val list_to_seq : 'a list -> 'a Seq.t = <fun>


val to_seq : 'a tree -> 'a Seq.t = <fun>


val solutions : clause list -> string -> (term list * term list) Seq.t =
  <fun>


val request : clause list -> string -> unit = <fun>


In [27]:
(* request world "etudiant_de(E, pierre)";;

request world "etudiant_de(E, pierre), etudiant_de(E, alice)";;

request world "etudiant_de(A, B)";;

request world "etudiant_de(A, A)";;

request world "apprend(A, A)";;

request world "enseigne(A, A)";;

request world "enseigne(alice, physique)";;

request world "enseigne(alice, mathematiques)";; *)

In [28]:
(* # load "prolog.cmo";;
open Prolog;; *)

In [29]:
let program = "
connected(bond_street,oxford_circus,central).
connected(oxford_circus,tottenham_court_road,central).
connected(bond_street,green_park,jubilee).
connected(green_park,charing_cross,jubilee).
connected(green_park,piccadilly_circus,piccadilly).
connected(piccadilly_circus,leicester_square,piccadilly).
connected(green_park,oxford_circus,victoria).
connected(oxford_circus,piccadilly_circus,bakerloo).
connected(piccadilly_circus,charing_cross,bakerloo).
connected(tottenham_court_road,leicester_square,northern).
connected(leicester_square,charing_cross,northern).

nearby(X,Y):-connected(X,Y,L).
nearby(X,Y):-connected(X,Z,L),connected(Z,Y,L).

reachable(X,Y):-connected(X,Y,L).
reachable(X,Y):-connected(X,Z,L),reachable(Z,Y).

path(X,Y,noroute):-connected(X,Y,L).
path(X,Y,route(Z,R)):-connected(X,Z,L),path(Z,Y,R).";; (*connected(X,Y,L):-connected(Y,X,L).*)

val program : string =
  "\nconnected(bond_street,oxford_circus,central).\nconnected(oxford_circus,tottenham_court_road,central).\nconnected(bond_street,green_park,jubilee).\nconnected(green_park,charing_cross,jubilee).\nconnected(green_park,piccadilly_circus,piccadilly).\nconnected(piccadilly_circus,leicester_square,piccadilly)."... (* string length 816; truncated *)


In [30]:
let world = parser program;;

val world : clause list =
  [Clause
    (Term ("connected",
      [Term ("bond_street", []); Term ("oxford_circus", []);
       Term ("central", [])]),
    []);
   Clause
    (Term ("connected",
      [Term ("oxford_circus", []); Term ("tottenham_court_road", []);
       Term ("central", [])]),
    []);
   Clause
    (Term ("connected",
      [Term ("bond_street", []); Term ("green_park", []);
       Term ("jubilee", [])]),
    []);
   Clause
    (Term ("connected",
      [Term ("green_park", []); Term ("charing_cross", []);
       Term ("jubilee", [])]),
    []);
   Clause
    (Term ("connected",
      [Term ("green_park", []); Term ("piccadilly_circus", []);
       Term ("piccadilly", [])]),
    []);
   Clause
    (Term ("connected",
      [Term ("piccadilly_circus", []); Term ("leicester_square", []);
       Term ("piccadilly", [])]),
    []);
   Clause
    (Term ("connected",
      [Term ("green_park", []); Term ("oxford_circus", []);
       Term ("victoria", [])]),
    []);
   Cla

In [31]:
request world "connected(piccadilly_circus,leicester_square,piccadilly)";;

This is True.


- : unit = ()


In [32]:
request world "nearby(oxford_circus, charing_cross)";;

This is True.


- : unit = ()


In [33]:
request world "nearby(tottenham_court_road,W)";;

There is : W = leicester_square
There is : W = charing_cross


- : unit = ()


In [34]:
request world "reachable(bond_street, leicester_square)";;

This is True.
This is True.
This is True.
This is True.
This is True.


- : unit = ()


In [35]:
request world "connected(oxford_circus, bond_street, L)";;

This is false.


- : unit = ()


In [36]:
request world "path(oxford_circus,charing_cross,R)";;

There is : R = route(tottenham_court_road, route(leicester_square, noroute))
There is : R = route(piccadilly_circus, noroute)
There is : R = route(piccadilly_circus, route(leicester_square, noroute))


- : unit = ()
